In [ ]:
import torch.nn as nn
import torchvision
import torch
import torchvision.models as models
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from torchvision.models import ResNet101_Weights,  MobileNet_V3_Large_Weights, EfficientNet_V2_S_Weights
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

print(f'Torch-Version {torch.__version__}')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'DEVICE: {DEVICE}')

#### **Reload pre-trained base model**

Reload the base models for further feature extraction.

In [ ]:
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNetFeatureExtractor, self).__init__()
        
        # Load the ResNet-101 model with pretrained weights
        resnet101 = models.resnet101(weights=ResNet101_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(resnet101.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class MobileNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(MobileNetFeatureExtractor, self).__init__()
        
        # Load the MobileNet model with pretrained weights
        Mobile_l = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(Mobile_l.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class EfficientNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetFeatureExtractor, self).__init__()
        
        # Load the EfficientNet model with pretrained weights
        Efficient = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(Efficient.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

# Define the base model structure
class SimCLR(nn.Module):
    def __init__(self, linear_eval=False):
        super().__init__()
        self.linear_eval = linear_eval
        feature_extractor = ResNetFeatureExtractor() # choose corresponding base model
        self.encoder = feature_extractor
        self.projection = nn.Sequential( 
            nn.Linear(2048, 128), # modify the input feature dimension based on models: resnet-2048, mobilenet-960, efficientnet-1024, add more models and modify them correctly
            nn.ReLU(),
            nn.Linear(128,128)
        )
    def forward(self, x):
        if not self.linear_eval:
            x = torch.cat(x, dim=0)
    
        encoding = self.encoder(x)
        projection = self.projection(encoding) 
        return projection

In [ ]:
# Define a new base model with the same structure but replaced with a new classification head.
class LinearEvaluation(nn.Module):
    def __init__(self, model, nu_classes):
        super().__init__()
        simclr = model
        simclr.linear_eval = True
        simclr.projection = Identity()
        self.simclr = simclr
        for param in self.simclr.parameters():
            param.requires_grad = False
        self.linear = nn.Sequential(nn.Linear(2048, 128), nn.ReLU()) # modify the input dimension to match the base model structure
        self.linear2 = nn.Sequential(nn.Linear(128, nu_classes), nn.Softmax(dim=-1))
    def forward(self, x):
        encoding = self.simclr(x)
        encoding = self.linear(encoding) 
        pred = self.linear2(encoding)
        return pred

In [ ]:
# Load trained base models
new_effici_model = torch.load('saved_model/pre-trained_model.pth')
new_resnet_model = torch.load('saved_model/pre-trained_model.pth')
new_mobile_model = torch.load('saved_model/pre-trained_model.pth')

#### **Feature extraction and fusion**

Place hooks to the base model and extract features from training and testing data.

In [ ]:
# placeholder for batch features
features = {}

def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()

    return hook

##### REGISTER HOOK

new_resnet_model.linear.register_forward_hook(get_features("resnet_feats"))
new_mobile_model.linear.register_forward_hook(get_features("mobile_feats"))
new_effici_model.linear.register_forward_hook(get_features("effici_feats"))

*Training data feature extraction and fusion*

In [ ]:
# Set arraies to store the extracted features and ground_truth labels
ResNet_FEATS = []
MobileNet_FEATS = []
EfficiNet_FEATS = []
Train_Label = []

# Feature extraction from training set
with torch.no_grad():
        for i, element in enumerate(train_dl): # Acquire the target train dataloader from data preprocessing file
                image, label = element
                image = image.to(DEVICE)
                label = label.to(DEVICE)
                resnet_pred = new_resnet_model(image)
                mobile_pred = new_mobile_model(image)
                effici_pred = new_effici_model(image)

                Train_Label.append(label.cpu().numpy())
                ResNet_FEATS.append(features["resnet_feats"].cpu().numpy())
                MobileNet_FEATS.append(features["mobile_feats"].cpu().numpy())
                EfficiNet_FEATS.append(features["effici_feats"].cpu().numpy())

In [ ]:
# Concatenate base model's extracted feature into a single array
ResNet_FEATS_Con = []
MobileNet_FEATS_Con = []
EfficiNet_FEATS_Con = []

ResNet_FEATS_Con = np.concatenate(ResNet_FEATS)
MobileNet_FEATS_Con = np.concatenate(MobileNet_FEATS)
EfficiNet_FEATS_Con = np.concatenate(EfficiNet_FEATS)

print("- feats shape:", ResNet_FEATS_Con.shape)
print("- feats shape:", MobileNet_FEATS_Con.shape)
print("- feats shape:", EfficiNet_FEATS_Con.shape)

Train_Label = np.concatenate(Train_Label, axis=0)

print("Labels shape:", Train_Label.shape)

In [ ]:
# Concatenate all features from base model
FEATS = []

FEATS = np.concatenate((ResNet_FEATS_Con, MobileNet_FEATS_Con, EfficiNet_FEATS_Con), axis=1)
print("- preds shape:", FEATS.shape)

*Testing data feature extraction and fusion*

In [ ]:
# Set arraies to store the extracted features and ground_truth labels
ResNet_Test_FEATS = []
MobileNet_Test_FEATS = []
EfficiNet_Test_FEATS = []
Test_Label = []

# Feature extraction from testing set
with torch.no_grad():
        for i, element in enumerate(test_dl):  # Acquire the target test dataloader from data preprocessing file
                image, label = element
                image = image.to(DEVICE)
                label = label.to(DEVICE)
                resnet_pred = new_resnet_model(image)
                mobile_pred = new_mobile_model(image)
                effici_pred = new_effici_model(image)

                Test_Label.append(label.cpu().numpy())
                ResNet_Test_FEATS.append(features["resnet_feats"].cpu().numpy())
                MobileNet_Test_FEATS.append(features["mobile_feats"].cpu().numpy())
                EfficiNet_Test_FEATS.append(features["effici_feats"].cpu().numpy())

In [ ]:
# Concatenate base model's extracted feature into a single array
ResNet_Test_FEATS_Con = []
MobileNet_Test_FEATS_Con = []
EfficiNet_Test_FEATS_Con = []

ResNet_Test_FEATS_Con = np.concatenate(ResNet_Test_FEATS)
MobileNet_Test_FEATS_Con = np.concatenate(MobileNet_Test_FEATS)
EfficiNet_Test_FEATS_Con = np.concatenate(EfficiNet_Test_FEATS)

print("- feats shape:", ResNet_Test_FEATS_Con.shape)
print("- feats shape:", MobileNet_Test_FEATS_Con.shape)
print("- feats shape:", EfficiNet_Test_FEATS_Con.shape)

Test_Label = np.concatenate(Test_Label, axis=0)

print("Labels shape:", Test_Label.shape)

In [ ]:
# Concatenate all features from base model
Test_FEATS = []

Test_FEATS = np.concatenate((ResNet_Test_FEATS_Con, MobileNet_Test_FEATS_Con, EfficiNet_Test_FEATS_Con), axis=1)
print("- preds shape:", Test_FEATS.shape)

#### **Feature selection**

Using Independent Component Analysis to conduct feature selection

In [ ]:
from sklearn.decomposition import FastICA

# Initialise the ICA model
ica = FastICA(n_components=50) # Change the n_components to the number of features that want to keep
ica.fit(FEATS)

X_train_ica = ica.transform(FEATS)
X_test_ica = ica.transform(Test_FEATS)

X_train_ica.shape

#### **Ensemble learning**

Use processed training features as the input data to train the higher-level ML classifier for ensemble learning, and use processed testing features to test the performance.


In [ ]:
#Initilise with SVM classifier
from sklearn.svm import SVC, LinearSVC

# Train SVM model
svm_model = LinearSVC(dual="auto", random_state=0, tol=1e-2)
svm_model.fit(X_train_ica, np.ravel(Train_Label, order='C')) 
svm_predictions = svm_model.predict(X_test_ica)

svm_accuracy = accuracy_score(Test_Label, svm_predictions)
svm_precision, svm_recall, svm_f1_score, _ = precision_recall_fscore_support(Test_Label, svm_predictions, average=None)

#Evaluate the predictions
print("SVM Accuracy:", svm_accuracy)
print("SVM recall:", svm_recall)
print("SVM precision:", svm_precision)
print("SVM f1:", svm_f1_score)

In [ ]:
from xgboost import XGBClassifier

# Train XGBoost model
xgb_model = XGBClassifier(booster = 'gbtree', eta = 0.9, max_depth = 10)
xgb_model.fit(X_train_ica, Train_Label)
# Use the trained XGBoost model to make predictions on the test dataset
xgb_predictions = xgb_model.predict(X_test_ica)

xgb_accuracy = accuracy_score(Test_Label, xgb_predictions)
xgb_precision, xgb_recall, xgb_f1_score, _ = precision_recall_fscore_support(Test_Label, xgb_predictions, average=None)

#Evaluate the predictions
print("XGBoost Accuracy:", xgb_accuracy)
print("XGBoost recall:", xgb_recall)
print("XGBoost precision:", xgb_precision)
print("XGBoost f1:", xgb_f1_score)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train RandomForest Classifier
rf_model = RandomForestClassifier(min_samples_split=3, max_features='log2')
rf_model.fit(X_train_ica, Train_Label)
rf_predictions = rf_model.predict(X_test_ica)

rf_accuracy = accuracy_score(Test_Label, rf_predictions)
rf_precision, rf_recall, rf_f1_score, _ = precision_recall_fscore_support(Test_Label, rf_predictions, average=None)

#Evaluate the predictions
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest recall:", rf_recall)
print("Random Forest precision:", rf_precision)
print("Random Forest f1:", rf_f1_score)

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Train Naive Bayes Classifier
nb_model = GaussianNB()
nb_model.fit(X_train_ica, Train_Label)
nb_predictions = nb_model.predict(X_test_ica)

nb_accuracy = accuracy_score(Test_Label, nb_predictions)
nb_precision, nb_recall, nb_f1_score, _ = precision_recall_fscore_support(Test_Label, nb_predictions, average=None)

#Evaluate the predictions
print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes recall:", nb_recall)
print("Naive Bayes precision:", nb_precision)
print("Naive Bayes f1:", nb_f1_score)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train k-Nearest Neighbors Classifier
knn_model = KNeighborsClassifier(n_neighbors=3, weights = 'distance', p=1)
knn_model.fit(X_train_ica, Train_Label)
knn_predictions = knn_model.predict(X_test_ica)

knn_accuracy = accuracy_score(Test_Label, knn_predictions)
knn_precision, knn_recall, knn_f1_score, _ = precision_recall_fscore_support(Test_Label, knn_predictions, average=None)

#Evaluate the predictions
print("KNN Accuracy:", knn_accuracy)
print("KNN recall:", knn_recall)
print("KNN precision:", knn_precision)
print("KNN f1:", knn_f1_score)

In [ ]:
#Conduct majority voting with trained ML classifiers
from sklearn.ensemble import VotingClassifier

# Combine predictions into a list
all_predictions = [nb_predictions, xgb_predictions, rf_predictions, svm_predictions, knn_predictions]

# Initialize a voting classifier using hard voting (majority voting)
voting_clf = VotingClassifier(estimators=[
    ('naive_bayes', nb_model),
    ('xgboost', xgb_model),
    ('rf', rf_model),
    ('svm', svm_model),
    ('knn', knn_model)
], voting='hard')

# Reshape predictions to ensure they have at most two dimensions
reshaped_predictions = [pred.reshape(-1, 1) for pred in all_predictions]

# Fit the voting classifier on the reshaped predictions
voting_clf.fit(np.hstack(reshaped_predictions), Test_Label)

# Make predictions using the voting classifier
ensemble_predictions = voting_clf.predict(np.hstack(reshaped_predictions))

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(Test_Label, ensemble_predictions)
ensemble_precision, ensemble_recall, ensemble_f1_score, _ = precision_recall_fscore_support(Test_Label, ensemble_predictions, average=None)

print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble recall:", ensemble_recall)
print("Ensemble precision:", ensemble_precision)
print("Ensemble f1:", ensemble_f1_score)

#### **Confusion matrix evaluation**

Evaluate the final predictions with ground truth label, and provide a confusion matrix.

In [ ]:
def cm_plt(ax, cm, classes, cmap, title, normalize, font_size= 16 ):
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes,
        yticklabels=classes,
        title=title,
        ylabel="True label",
        xlabel="Predicted label",
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=font_size)
    plt.setp(ax.get_yticklabels(), fontsize=font_size)
    # Loop over data dimensions and create text annotations.
    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], fmt),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
                fontsize=font_size
            )

    return ax

# Defining function for confusion matrix plot
def plt_confusion_mat(cm, classes, fig_size, cmap=plt.cm.Blues, font_size=16):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=fig_size)
    ax1 = cm_plt(
        ax1,
        cm,
        classes,
        cmap,
        title="Confusion matrix, without normalization",
        normalize=False,
        font_size=font_size
    )

    cmn = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    ax2 = cm_plt(
        ax2,
        cmn,
        classes,
        cmap,
        title="Normalized confusion matrix",
        normalize=True,
        font_size=font_size
    )

In [ ]:
from sklearn.metrics import confusion_matrix
# Store class label in sequence of saving data 
Label = ['x','xx'] 
# A sample of Kvasirv2 dataset
Kvasirv2_label = ['Dyed-lifted-polyps', 'Dyed-resection-margins', 'Esophagitis', 'Normal-cecum', 'Normal-pylorus', 'Normal-z-line', 'Polyps', 'Ulcerative-colitis']

# Convert predicted_results and ground_truth_labels to NumPy arrays
predicted_results = np.array(ensemble_predictions)
ground_truth_labels = np.array(Test_Label)

# Calculate the confusion matrix
cm = confusion_matrix(predicted_results, ground_truth_labels)

# Call the confusion matrix plotting functions
fig_size = (24, 7)  # Adjust the figure size as needed
plt_confusion_mat(cm, Label, fig_size)

#### **t-SNE feature analysis**

Evaluate the whole model's feature classification quality.

In [ ]:
from sklearn.manifold import TSNE

# Assuming you have concatenated the feature maps from the base models into a single array called 'fused_features'

# Perform dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
embedded_features = tsne.fit_transform(X_test_ica)

# Visualize the embedded features
plt.figure(figsize=(10, 8))
plt.scatter(embedded_features[:, 0], embedded_features[:, 1], c=Train_Label, cmap='viridis')
plt.colorbar(label='Classes')
plt.title('t-SNE Visualization of Fused Features')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')

Kvasirv2_label = ['Dyed-lifted-polyps', 'Dyed-resection-margins', 'Esophagitis', 'Normal-cecum', 'Normal-pylorus', 'Normal-z-line', 'Polyps', 'Ulcerative-colitis']

# Add label names as annotations
for i, label in enumerate(Kvasirv2_label):
    plt.annotate(label, (embedded_features[Train_Label == i, 0].mean(), embedded_features[Train_Label == i, 1].mean()),
                 ha='center', va='center', size=10, bbox=dict(facecolor='white', alpha=0.5))

plt.show()